In [1]:
import os
from typing import List, Dict
from dotenv import load_dotenv

from langfuse import Langfuse
import pandas as pd
import tiktoken

from config.base_config import rag_config

In [2]:
load_dotenv()

LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY", None)
LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY", None)
LANGFUSE_HOST = "http://localhost:3000"

In [3]:
langfuse = Langfuse(
  secret_key=LANGFUSE_SECRET_KEY,
  public_key=LANGFUSE_PUBLIC_KEY,
  host=LANGFUSE_HOST
)

In [4]:
tokenizer = tiktoken.get_encoding("o200k_base")

In [5]:
pricing = {
    "gpt-4o": {
        "input": 5,
        "output": 15
    },
    "gpt-4o-2024-08-06": {
        "input": 2.5,
        "output": 10
    },
    "gpt-4o-2024-05-13": {
        "input": 5,
        "output": 15
    },
    "gpt-4o-mini": {
        "input": 0.15,
        "output": 0.6
    },
    "gpt-4o-mini-2024-07-18": {
        "input": 0.15,
        "output": 0.6
    },
    "chatgpt-4o-latest": {
        "input": 5.00,
        "output": 15.00
    },
    "gpt-4-turbo": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-4-turbo-2024-04-09": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-4": {
        "input": 30.00,
        "output": 60.00
    },
    "gpt-4-32k": {
        "input": 60.00,
        "output": 120.00
    },
    "gpt-4-0125-preview": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-4-1106-preview": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-4-vision-preview": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-3.5-turbo-0125": {
        "input": 0.50,
        "output": 1.50
    },
    "gpt-3.5-turbo-instruct": {
        "input": 1.50,
        "output": 2.00
    },
    "gpt-3.5-turbo-1106": {
        "input": 1.00,
        "output": 2.00
    },
    "gpt-3.5-turbo-0613": {
        "input": 1.50,
        "output": 2.00
    },
    "gpt-3.5-turbo-16k-0613": {
        "input": 3.00,
        "output": 4.00
    },
    "gpt-3.5-turbo-0301": {
        "input": 1.50,
        "output": 2.00
    }
 }

In [6]:
model = rag_config["llm"]["model"]

if model in ["gpt-4o", "gpt-4o-2024-05-13", "gpt-4o-2024-08-06", "chatgpt-4o-latest", "gpt-4o-mini", "gpt-4o-mini-2024-07-18"]:
    encoding = "o200k_base"
elif model in ["gpt-4-turbo", "gpt-4-turbo-2024-04-09", "gpt-4-turbo-preview", "gpt-4-0125-preview", "gpt-4-1106-preview", "gpt-4",
               "gpt-4-0613", "gpt-4-0314", "gpt-3.5-turbo-0125", "gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-3.5-turbo-instruct"]:
    encoding = "cl100k_base"

tokenizer = tiktoken.get_encoding(encoding)

def get_cost(tokenizer, input: List[str], output: List[str], pricing: Dict, model: str):

    n_input_toks = len(tokenizer.encode(input))
    n_output_toks = len(tokenizer.encode(output))
    input_cost = n_input_toks * pricing[model]["input"] / 1_000_000
    output_cost = n_input_toks * pricing[model]["output"] / 1_000_000

    return input_cost + output_cost

### Get traces

In [7]:
traces = langfuse.fetch_traces().data

In [ ]:
trace_data = []

for trace in traces:

    input = trace.input["args"][1]["query"]
    output = "".join(trace.output) if trace.output else ""
    trace_data.append(
        {
            "id": trace.id,
            "timestamp": trace.timestamp.strftime('%Y-%m-%d %H:%M:%S'),
            "latency": trace.latency,
            "cost": get_cost(tokenizer=tokenizer,
                       input=input,
                       output=output,
                       pricing=pricing,
                       model=model),
            "input": input,
            "output": output
        }
    )

trace_data_df = pd.DataFrame(trace_data)
trace_data_df

In [ ]:
trace_data_df.cost.sum()

In [ ]:
trace_data_df.describe()

### Observations

In [30]:
observations = langfuse.fetch_observations(name="retrieve")

In [38]:
obs = {obs.trace_id: obs.output for obs in observations.data}
trace_data_df["retrieval"] = trace_data_df["id"].map(obs)

In [ ]:
trace_data_df